## Set Up

In [ ]:
# Prepare the environment


%cd /datasets/drive/python-library
!pip install .
!sudo apt-get install python-pymysql
!pip install colab-env
!pip install Hashids
%cd /datasets/drive
from datetime import datetime
import pytz
import nltk
nltk.download('stopwords')
import gallifrey
import pandas as pd
import numpy as np
import time
from hashids import Hashids

#Set up for Hash calculation
hasher = Hashids("jS5qAlR0kHbAeqeuVowPBnyotKuTvvlGzIzkEtz79XQGB7BJHb", 
                 8, 
                 "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ1234567890")

# Gspread

import gspread
gc = gspread.service_account(filename='/work/credentials.json')
# create, and save df
from gspread_dataframe import set_with_dataframe

# Import environment varibles
import os

gallifrey.discovery.ssh_pkey = os.getenv('BLACK_PLATINUM_KEY_PATH')
gallifrey.solar.ssh_pkey = os.getenv('BLACK_PLATINUM_KEY_PATH')

/datasets/drive/.shared/python-library
Processing /datasets/drive/.shared/python-library
  Created wheel for gallifrey: filename=gallifrey-0.3.0-py3-none-any.whl size=9864 sha256=161068785dd8a2190f26e6db92bf8859d68e973af5b4e21944b6a4f292709ded
  Stored in directory: /root/.cache/pip/wheels/de/b7/e0/b51405f454f78fc430c6037b1542f1ad52fd08b2e643ab25fa
Successfully built gallifrey
  Attempting uninstall: gallifrey
    Found existing installation: gallifrey 0.3.0
    Uninstalling gallifrey-0.3.0:
      Successfully uninstalled gallifrey-0.3.0
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.



E: Unable to locate package python-pymysql
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
/datasets/drive
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpor

## Auto Update

#### Query


In [ ]:
!rm -r Job-listing-Query
!git clone --branch Job-listing-3.0  https://github.com/alexisnpavlik/Job-listing-Query.git

Cloning into 'Job-listing-Query'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (449/449), done.
remote: Compressing objects: 100% (310/310), done.
remote: Total 449 (delta 265), reused 313 (delta 137), pack-reused 0
Receiving objects: 100% (449/449), 72.27 KiB | 113.00 KiB/s, done.
Resolving deltas: 100% (265/265), done.
Checking out files: 100% (11/11), done.


In [ ]:
main_query=open("""Job-listing-Query/main_query.sql""", 'r').read()
organizations_query=open("""Job-listing-Query/organizations_query.sql""", 'r').read()
notifications_query=open("""Job-listing-Query/notifications.sql""", 'r').read()
request_manual_invite_query=open("""Job-listing-Query/request_manual_invite.sql""", 'r').read()
invited_from_suggested_query=open("""Job-listing-Query/invited_from_suggested.sql""", 'r').read()
total_suggested_query=open("""Job-listing-Query/total_suggested.sql""", 'r').read()
commited_query=open("""Job-listing-Query/commited.sql""", 'r').read()
response_times_query=open("""Job-listing-Query/response_times.sql""", 'r').read()
aplications_query=open("""Job-listing-Query/aplications.sql""", 'r').read()
request_automated_sourcing_query=open("""Job-listing-Query/request_automated_sourcing.sql""", 'r').read()

In [ ]:
start_time=time.time()
main = pd.DataFrame(gallifrey.discovery.execute_query(main_query))
organizations = pd.DataFrame(gallifrey.solar.execute_query(organizations_query))
notifications = pd.DataFrame(gallifrey.snowplow.execute_query(notifications_query))
request_manual_invite = pd.DataFrame(gallifrey.solar.execute_query(request_manual_invite_query))
invited_from_suggested = pd.DataFrame(gallifrey.discovery.execute_query(invited_from_suggested_query))
total_suggested = pd.DataFrame(gallifrey.discovery.execute_query(total_suggested_query))
commited = pd.DataFrame(gallifrey.discovery.execute_query(commited_query))
response_times = pd.DataFrame(gallifrey.discovery.execute_query(response_times_query))
aplications = pd.DataFrame(gallifrey.discovery.execute_query(aplications_query))
request_automated_sourcing = pd.DataFrame(gallifrey.solar.execute_query(request_automated_sourcing_query))

hubspot= pd.read_csv('/datasets/drive/Datasets/all_tickets_hs.csv')

print('Tiempo de ejecución:',time.time()-start_time,'s')

Tiempo de ejecución: 190.0879111289978 s


### Copy

In [ ]:
main_df = main.copy(deep=True)
organizations_df = organizations.copy(deep=True)
notifications_df = notifications.copy(deep=True)
request_manual_invite_df = request_manual_invite.copy(deep=True)
invited_from_suggested_df = invited_from_suggested.copy(deep=True)
total_suggested_df = total_suggested.copy(deep=True)
commited_df = commited.copy(deep=True)
response_times_df = response_times.copy(deep=True)
aplications_df = aplications.copy(deep=True)
request_automated_sourcing_df = request_automated_sourcing.copy(deep=True)

hubspot_df = hubspot.copy(deep=True)


In [ ]:
index = request_automated_sourcing_df.index[request_automated_sourcing_df['AlfaID'].str.startswith('https://torre.co/postings/')].tolist()
request_automated_sourcing_df.drop(index, axis = 0, inplace=True)

In [ ]:
# Alfa ID Generator
def alfa_id(id):
  return hasher.encode(id)

# URL Generator
def url_generator(id):
  return 'https://torre.co/postings/'+hasher.encode(id)

# Manager URL Generator
def manager_generator(id):
  return 'https://torre.co/postings/'+hasher.encode(id)+'/members-list?sharingToken='

# Transforms columns
main_df.insert(1,"Alfa ID", main_df['ID'].apply(alfa_id),True)
main_df.insert( 2,"URL", main_df['ID'].apply(url_generator),True)

main_df.insert( 6,"Manager link", 'https://torre.co/postings/'+ main_df['Alfa ID'] +'/members-list?sharingToken='+ main_df['Sharing token'] ,True)

# Last update Generator
my_city_timezone = pytz.timezone('America/Bogota')
my_city_time = datetime.now(my_city_timezone)
update_hour = my_city_time.strftime("%H:%M - %d/%m (GMT-5)")
main_df.insert( 2,"Last Update", update_hour,True)

# URL Hubspot Generator
def hubspot_generator(id):
  return 'https://app.hubspot.com/contacts/5943984/ticket/'+str(id)

hubspot_df.insert( 2,"Hubspot URL", hubspot_df['HS_ID'].apply(hubspot_generator),True)

# Hubspot name Generator
def name_generator(id):
    if id == 82061645:
        return 'Natalia Arrieta Aycardi'
    elif id == 101607757:
        return 'Juan Pablo Santa'
    elif id == 64896811:
        return 'Santiago Reyes'
    elif id == 42228953:
        return 'Daniela Botero'
    elif id == 67362916:
        return 'Jhon Marin'
    elif id == 110277165:
        return 'Lubianca Vanegas'
    elif id == 141011802: 
        return 'Juan Manuel Salvadé'
    elif id == 72021367:
        return 'Melissa Erasso'
    elif id == 83083950:
        return 'raynier@torre.co'
    elif id == 39113169:
        return 'Maria Fernanda Mosquera'
    elif id == 83549794:
        return 'Mauricio Zabala'
    elif id == 110277130:
        return 'augustin@torre.co'
    elif id == 137166627:
        return 'Neider Fayad'
    elif id == 120967450:
        return 'Natalia Sánchez Rodríguez'
    elif id == 119544834:
        return 'Maria Isabel Silva Baquero'
    elif id == 118296335:
        return 'Daniela Bonilla'
    elif id == 128193536:
        return 'Tatiana Palomino'
    elif id == 99570339:
        return 'Sebastian Ruiz'
    elif id == 140100897:
        return 'Santiago Higuera'
    elif id == 127205203:
        return 'Wilson Peña'
    elif id == 138004282:
        return 'Julio César Espinosa Herrera'
    elif id == 140327503:
        return 'María Gabriela Donoso Delgado'
    elif id == 123190151:
        return 'Renzo Martínez'
    elif id == 87427407:
        return 'Alexandra Rodriguez'
    elif id == 134119974:
        return 'Angelo Vásquez'
    elif id == 128914705:
        return 'Laura Mojica'
    elif id == 108959025:
        return 'Carlos Lopera'
    elif id == 121379976:
        return 'Melissa Ricardo Hoyos'
    elif id == 119634958:
        return 'TATIANA VELEZ'
    elif id == 109029174:
        return 'M. Catalina Morales'
    elif id == 90384244:
        return 'Paula Marulanda'
    elif id == 108959023:
        return 'Ricardo Adolfo Saavedra Lozano'
    elif id == 97789475:
        return 'Federico Romero'
    elif id == 128914703:
        return 'Jose A. Herrera'
    elif id == 109029171:
        return 'Jose Berrio'
    elif id == 59231664:
        return 'Daniela Gaviria'
    elif id == 114478130:
       return 'Natalia Torres Casto' 
    elif id == 88416842:
        return 'Vanessa Castro'
    elif id == 84084140:
        return 'Andres Meza'
    elif id == 91698865:
        return 'Maria Alejandra Gómez'
    elif id == 93511743:
        return 'Darío Estupiñán'
    elif id == 84229495:
        return 'Alexis Najarro'
    elif id == 71114922:
        return 'María Juliana Pardo'
    elif id == 63666042: 
        return 'Nelly cacho'
    elif id == 59232375:
        return 'Natalia Rodriguez'
    elif id == 97135664:
        return 'Andrés Mauricio Jiménez García'
    elif id == 59232426:
            return 'Hermann Cardona'
    elif id == 134153486:
            return 'Sebastian Nieto'
    elif id == 101649043:
            return 'Alessio Vargas'
    elif id == 42833375:
            return 'Diana Montoya'
    elif id == 105037059:
            return 'Roy Villamizar'
    elif id == 42833375:
            return 'Renzo Martínez'
    elif id == 84924686:
            return 'Paula Corredor'
    elif id == 63666041:
            return 'Valentina Santos'
    elif id == 98133556:
            return 'Paco Ruiz'
    elif id == 39847960:
            return 'Diana Aparicio'
    elif id == 77712296:
            return 'Paula Katherine Lozano Molina'
    elif id == 54625855:
            return 'Susana Ayala'
    elif id == 138392444:
            return 'Angelo Vásquez'
    elif id == 97789474:
            return 'Jessica Pinzon'
    elif id == 70032248:
            return 'Eduardo Lombana'
    elif id == 90657168:
            return 'Ignacio Minenna'
    elif id == 149866887:
            return 'Zandokan Villarreal'
    elif id == 108502892:
            return 'Catalina González Salcedo'
    elif id == 62009144:
            return 'Carolina Gomez Villafane'
    elif id == 89350190:
            return 'Danna Melissa Munevar Hernandez'
    elif id == 113619525:
            return 'Laura Sánchez'
    elif id == 138861200:
            return 'Carlos Torres'
    elif id == 91737210:
            return 'Jose A. Herrera'
    elif id == 131285778:
            return 'Manuela Zuluaga'
    elif id == 107943755:
            return 'David Giraldo Valencia'
    


hubspot_df.insert( 2,"Ticket Owner", hubspot_df['Ticket Owner ID'].apply(name_generator),True)

main_df.Timezones = main_df.Timezones.replace({"""GMT-12:00
GMT+14:00""":'Remote Anywhere'})

In [ ]:
response_times_df = response_times_df.pivot(index='ID', columns='utm_medium', values='RT')

In [ ]:
notifications_df = notifications_df.rename(columns={'alfaid':'Alfa ID','realistic':'Sent Matches','people':'Sent SigPeop','organization': 'Sent SigOrg'})
request_manual_invite_df = request_manual_invite_df.rename(columns={'AlfaID':'Alfa ID','sent':'Sent manual invite','pending':'Pending manual invite','canceled':'Canceled manual invite'})
request_automated_sourcing_df = request_automated_sourcing_df.rename(columns={'AlfaID':'Alfa ID'})
response_times_df = response_times_df.rename(columns={'ja_mtc':'RT Job alerts from matches','ja_rlvsgl_prs':'RT Job alerts from relevant signals people','ja_allsgl_prs':'RT Job alerts from all signals people','ja_rlvsgl_org':'RT Job alerts from relevant signals organization','ja_allsgl_org':'RT Job alerts from all signals organization','rc_cb_rcdt':'RT Recruiter chatbot','rc_ccg':'RT Recruiter chatbot to CCGs','rc_syn':'RT Recruiter chatbot syndication','rc_src':'RT Recruiter chatbot sourcing','am_sug':'RT Auto triggered messages to suggested candidates','srh_jobs':'RT Job search','sml_jobs':'RT Similar jobs','am_inv':'RT Auto triggered messages to invited external candidates','google_jobs':'RT Google for jobs','fb_jobs':'RT Facebook jobs','ref_ts':'RT Referrals from the TS','shr_ts':'RT Talent seekers sharing their own post','ro_sug':'RT Personal reach out to manually invited','pr_sml_jobs':'RT Personal reach out to candidates of similar jobs','rc_trrx_inv':'RT Manual invite by Torrex of registered candidates (internal sourcing)','syn':'RT Syndication','src':'RT Sourcing','ref_ptn':'RT Referrals from partners','ref_cdt':'RT Referrals from candidates','ref_vst_imp':'RT Referrals from visitors that share a job outside Torre (implicitly)','syn_rqt':'RT Syndication as requested','trr_webinars':'RT Webinars Torre Access'})

In [ ]:
opportunities_df = pd.merge(left=main_df,right=organizations_df,how='left', on='Company_id')
opportunities_df = pd.merge(left=opportunities_df,right=notifications_df, how='left', on='Alfa ID')
opportunities_df = pd.merge(left=opportunities_df,right=request_manual_invite_df, how='left', on='Alfa ID')
opportunities_df = pd.merge(left=opportunities_df,right=invited_from_suggested_df, how='left', on='ID')
opportunities_df = pd.merge(left=opportunities_df,right=total_suggested_df, how='left', on='ID')
opportunities_df = pd.merge(left=opportunities_df,right=commited_df, how='left', on='ID')
opportunities_df = pd.merge(left=opportunities_df,right=aplications_df, how='left', on='ID')
opportunities_df = pd.merge(left=opportunities_df,right=response_times_df, how='left', on='ID')
opportunities_df = pd.merge(left=opportunities_df,right=request_automated_sourcing_df, how='left', on='Alfa ID')
opportunities_df = pd.merge(left=opportunities_df,right=hubspot_df, how='left', on='ID')


opportunities_df = opportunities_df.reindex(columns=['ID','Job title','Company','Alfa ID','Location','Timezones','Created date','Approved date','Commited date','Applicant Acquisition Coordinator','Type of job',
                                             'Type of service','URL','Manager link','Hubspot URL','Ticket Owner ID','Ticket Owner','Sourcer','Syndicator','Number of Syndication post','Contacted from Manual Sourcing','Search Results','Waterfall Channels','Timestamp waterfall channels','Feedback superfilter','Feedback red opp','Feedback dry opp','Language of the post','Status','Reason','Last changes','Closing Date','Closed date','Outbound Efforts','Last Update',
                                             'Completed applications','Completed applications yesterday','Completed applications in the last 14 days','Incomplete applications','Mutual matches','Real Active','Opportunity approved yesterday','Disqualified','Others','Hires','Hires yesterday',
                                             'Sent Matches','Sent SigPeop','Sent SigOrg','Sent manual invite','Pending manual invite','Canceled manual invite','Invited from suggested','Suggested','Requests from automated sourcing' ,'RT Job alerts from matches','RT Job alerts from relevant signals people','RT Job alerts from all signals people','RT Job alerts from relevant signals organization','RT Job alerts from all signals organization','RT Recruiter chatbot','RT Recruiter chatbot to CCGs','RT Recruiter chatbot syndication','RT Recruiter chatbot sourcing','RT Auto triggered messages to suggested candidates','RT Job search','RT Similar jobs','RT Auto triggered messages to invited external candidates','RT Google for jobs','RT Facebook jobs','RT Referrals from the TS','RT Talent seekers sharing their own post','RT Personal reach out to manually invited','RT Personal reach out to candidates of similar jobs','RT Manual invite by Torrex of registered candidates (internal sourcing)','RT Syndication','RT Sourcing','RT Referrals from partners','RT Referrals from candidates','RT Referrals from visitors that share a job outside Torre (implicitly)','RT Syndication as requested','RT Webinars Torre Access', 'Job alerts from matches','Job alerts from relevant signals people','Job alerts from all signals people','Job alerts from relevant signals organization','Job alerts from all signals organization','Recruiter chatbot','Recruiter chatbot to CCGs','Recruiter chatbot syndication','Recruiter chatbot sourcing','Auto triggered messages to suggested candidates','Job search','Similar jobs','Auto triggered messages to invited external candidates','Google for jobs','Facebook jobs','Referrals from the TS','Talent seekers sharing their own post','Personal reach out to manually invited','Personal reach out to candidates of similar jobs','Manual invite by Torrex of registered candidates (internal sourcing)','Syndication','Sourcing','Referrals from partners','Referrals from candidates','Referrals from visitors that share a job outside Torre (implicitly)','Syndication as requested','Webinars Torre Access'])

In [ ]:
sheet = gc.open_by_url('https://docs.google.com/spreadsheets/d/10Exx43oNevJzeV6FTgKxJSe-oXMFoS300826lsJkYrE/edit#gid=0').sheet1

sheet.clear()

set_with_dataframe(sheet, opportunities_df) 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5821676f-5400-4fde-a896-01206472ffab' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>